In [1]:
import numpy as np
import pandas as pd
import scipy
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

%matplotlib inline
from matplotlib import pyplot as plt

In [2]:
# при необходимости поменяйте путь к данным 
train_df = pd.read_csv('C:\\programmer_shit\\data_salo_files\\habr\\howpop_train.csv')
test_df  = pd.read_csv('C:\\programmer_shit\\data_salo_files\\habr\\howpop_test.csv')



In [3]:
train_df

,url,domain,post_id,published,author,flow,polling,content_len,title,comments,favs,views,votes_plus,votes_minus,views_lognorm,favs_lognorm,comments_lognorm
0,https://habrahabr.ru/post/18284/,habrahabr.ru,18284,2008-01-01 18:19:00,@Tapac,develop,False,4305,Новогодний подарок блоггерам — WordPress 2.3.2,0,0,236,0.0,0.0,-0.792687,-1.344075,-2.436871
1,https://habrahabr.ru/post/18285/,habrahabr.ru,18285,2008-01-01 18:30:00,@DezmASter,design,False,7344,"Сумасшедшие яйца, или сервис для отслеживания ...",1,1,353,1.0,0.0,-0.485214,-0.831946,-1.873193
2,https://habrahabr.ru/post/18286/,habrahabr.ru,18286,2008-01-01 18:34:00,@DezmASter,design,False,8431,"Сумасшедшие яйца, или сервис для отслеживания ...",47,72,1200,45.0,5.0,0.446090,1.794023,0.574656
3,https://habrahabr.ru/post/18291/,habrahabr.ru,18291,2008-01-02 01:32:00,@Taoorus,design,False,5662,"Сглаживание шрифтов, и субпиксельная отрисовка",102,36,5700,72.0,3.0,1.627469,1.300050,1.167375
4,https://geektimes.ru/post/18294/,geektimes.ru,18294,2008-01-02 14:34:00,@dennydo,NaN,False,3706,Почему мне не нравится iPhone,230,6,1400,73.0,15.0,1.362397,0.961094,2.020356
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134132,https://habrahabr.ru/company/mailru/blog/314014/,habrahabr.ru,314014,2016-10-31 21:45:00,NaN,develop,False,56033,Парсинг JSON — это минное поле,58,347,35500,164.0,6.0,1.870174,2.124862,1.258447
134133,https://geektimes.ru/post/282086/,geektimes.ru,282086,2016-10-31 22:44:00,@marks,NaN,False,5504,Заработала система раннего предупреждения НАСА...,52,16,12900,22.0,0.0,-0.091464,-0.511814,0.465118
134134,https://geektimes.ru/post/282090/,geektimes.ru,282090,2016-10-31 23:26:00,@Shapelez,NaN,False,2883,Арзамасский городской суд постановил заблокиро...,16,14,15800,28.0,4.0,0.198924,-0.656390,-0.408714
134135,https://geektimes.ru/post/282092/,geektimes.ru,282092,2016-10-31 23:52:00,@ragequit,NaN,False,5735,Медиа-коммуникационный союз планирует внести в...,47,14,15000,20.0,0.0,0.124244,-0.655385,0.389636


In [4]:
test_df

,url,domain,post_id,published,author,flow,polling,content_len,title
0,https://habrahabr.ru/post/314080/,habrahabr.ru,314080,2016-11-01 01:05:00,@fsou11,develop,True,20132,Опыт использования MassTransit 3.0
1,https://habrahabr.ru/company/plesk/blog/313732/,habrahabr.ru,313732,2016-11-01 05:42:00,NaN,marketing,False,12389,Геймификация форума на движке XenForo
2,https://habrahabr.ru/company/etagi/blog/314000/,habrahabr.ru,314000,2016-11-01 08:33:00,NaN,admin,False,72839,Кластер высокой доступности на postgresql 9.6 ...
3,https://habrahabr.ru/company/knopka/blog/314030/,habrahabr.ru,314030,2016-11-01 09:00:00,NaN,develop,False,29572,Как перестать бояться и полюбить синтаксически...
4,https://geektimes.ru/company/audiomania/blog/2...,geektimes.ru,282058,2016-11-01 09:44:00,NaN,NaN,False,9642,Мифы и реальность: Что нужно знать о современн...
...,...,...,...,...,...,...,...,...,...
3985,https://geektimes.ru/post/281642/,geektimes.ru,281642,2017-01-26 09:40:00,@PatientZero,NaN,False,23613,История разработки Deus Ex
3986,https://habrahabr.ru/company/tuturu/blog/320326/,habrahabr.ru,320326,2017-01-26 09:43:00,NaN,develop,False,19504,"Все лучшее из методологии Lean Startup, и как ..."
3987,https://habrahabr.ru/company/gemaltorussia/blo...,habrahabr.ru,320214,2017-01-26 10:00:00,NaN,develop,False,8395,Защита больших данных – как правильно начать и...
3988,https://habrahabr.ru/post/320472/,habrahabr.ru,320472,2017-01-26 10:01:00,@Reason89,develop,False,7788,Подключение Xmega к FRAM по SPI


In [5]:
train_df.head(1).T

,0
url,https://habrahabr.ru/post/18284/
domain,habrahabr.ru
post_id,18284
published,2008-01-01 18:19:00
author,@Tapac
flow,develop
polling,False
content_len,4305
title,Новогодний подарок блоггерам — WordPress 2.3.2
comments,0


In [6]:
train_df.shape, test_df.shape

((134137, 17), (3990, 9))

In [7]:
cor = train_df.corr()
cor

,post_id,polling,content_len,comments,favs,views,votes_plus,votes_minus,views_lognorm,favs_lognorm,comments_lognorm
post_id,1.000000,0.081628,0.241384,-0.147763,0.089313,0.132385,-0.187885,-0.235594,-0.010376,-0.005845,-0.001605
polling,0.081628,1.000000,-0.003337,0.042605,0.006882,0.025660,-0.033505,0.027116,-0.003523,-0.036385,0.071417
content_len,0.241384,-0.003337,1.000000,-0.023544,0.308194,0.204101,0.068779,-0.078686,0.246063,0.356481,0.073132
comments,-0.147763,0.042605,-0.023544,1.000000,0.164166,0.290035,0.613961,0.457638,0.349568,0.278942,0.662740
favs,0.089313,0.006882,0.308194,0.164166,1.000000,0.634304,0.416241,0.062877,0.456097,0.587982,0.263239
views,0.132385,0.025660,0.204101,0.290035,0.634304,1.000000,0.396849,0.128654,0.585105,0.406782,0.326427
votes_plus,-0.187885,-0.033505,0.068779,0.613961,0.416241,0.396849,1.000000,0.464168,0.414232,0.449712,0.525081
votes_minus,-0.235594,0.027116,-0.078686,0.457638,0.062877,0.128654,0.464168,1.000000,0.146609,0.088226,0.355458
views_lognorm,-0.010376,-0.003523,0.246063,0.349568,0.456097,0.585105,0.414232,0.146609,1.000000,0.688811,0.546530
favs_lognorm,-0.005845,-0.036385,0.356481,0.278942,0.587982,0.406782,0.449712,0.088226,0.688811,1.000000,0.479476


0              post_id
1              polling
2          content_len
3             comments
4                 favs
5                views
6           votes_plus
7          votes_minus
8        views_lognorm
9         favs_lognorm
10    comments_lognorm
dtype: object

In [9]:
numbers_of_index = pd.Series(cor.index)
start_count = 0
len_matrix = len(numbers_of_index)
for i in range(start_count, len_matrix):
    for j in range(start_count + 1 , len_matrix):
        cur_value = cor.iloc[i][j]
        if cur_value > 0.9:
            print(numbers_of_index[i] , numbers_of_index[j])
            
    start_count +=1
        
        
        

In [10]:
train_df["published"] = pd.to_datetime(train_df["published"])

In [11]:
train_df.dtypes

url                         object
domain                      object
post_id                      int64
published           datetime64[ns]
author                      object
flow                        object
polling                       bool
content_len                  int64
title                       object
comments                     int64
favs                         int64
views                        int64
votes_plus                 float64
votes_minus                float64
views_lognorm              float64
favs_lognorm               float64
comments_lognorm           float64
dtype: object

In [12]:
train_df.set_index('published')
train_df

,url,domain,post_id,published,author,flow,polling,content_len,title,comments,favs,views,votes_plus,votes_minus,views_lognorm,favs_lognorm,comments_lognorm
0,https://habrahabr.ru/post/18284/,habrahabr.ru,18284,2008-01-01 18:19:00,@Tapac,develop,False,4305,Новогодний подарок блоггерам — WordPress 2.3.2,0,0,236,0.0,0.0,-0.792687,-1.344075,-2.436871
1,https://habrahabr.ru/post/18285/,habrahabr.ru,18285,2008-01-01 18:30:00,@DezmASter,design,False,7344,"Сумасшедшие яйца, или сервис для отслеживания ...",1,1,353,1.0,0.0,-0.485214,-0.831946,-1.873193
2,https://habrahabr.ru/post/18286/,habrahabr.ru,18286,2008-01-01 18:34:00,@DezmASter,design,False,8431,"Сумасшедшие яйца, или сервис для отслеживания ...",47,72,1200,45.0,5.0,0.446090,1.794023,0.574656
3,https://habrahabr.ru/post/18291/,habrahabr.ru,18291,2008-01-02 01:32:00,@Taoorus,design,False,5662,"Сглаживание шрифтов, и субпиксельная отрисовка",102,36,5700,72.0,3.0,1.627469,1.300050,1.167375
4,https://geektimes.ru/post/18294/,geektimes.ru,18294,2008-01-02 14:34:00,@dennydo,NaN,False,3706,Почему мне не нравится iPhone,230,6,1400,73.0,15.0,1.362397,0.961094,2.020356
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134132,https://habrahabr.ru/company/mailru/blog/314014/,habrahabr.ru,314014,2016-10-31 21:45:00,NaN,develop,False,56033,Парсинг JSON — это минное поле,58,347,35500,164.0,6.0,1.870174,2.124862,1.258447
134133,https://geektimes.ru/post/282086/,geektimes.ru,282086,2016-10-31 22:44:00,@marks,NaN,False,5504,Заработала система раннего предупреждения НАСА...,52,16,12900,22.0,0.0,-0.091464,-0.511814,0.465118
134134,https://geektimes.ru/post/282090/,geektimes.ru,282090,2016-10-31 23:26:00,@Shapelez,NaN,False,2883,Арзамасский городской суд постановил заблокиро...,16,14,15800,28.0,4.0,0.198924,-0.656390,-0.408714
134135,https://geektimes.ru/post/282092/,geektimes.ru,282092,2016-10-31 23:52:00,@ragequit,NaN,False,5735,Медиа-коммуникационный союз планирует внести в...,47,14,15000,20.0,0.0,0.124244,-0.655385,0.389636


In [13]:
train_df_dates = pd.read_csv('C:\\programmer_shit\\data_salo_files\\habr\\howpop_train.csv' , index_col='published',
                             parse_dates=True)
# train_df["published"] = pd.to_datetime(train_df["published"]
train_df_dates

,url,domain,post_id,author,flow,polling,content_len,title,comments,favs,views,votes_plus,votes_minus,views_lognorm,favs_lognorm,comments_lognorm
published,,,,,,,,,,,,,,,,
2008-01-01 18:19:00,https://habrahabr.ru/post/18284/,habrahabr.ru,18284,@Tapac,develop,False,4305,Новогодний подарок блоггерам — WordPress 2.3.2,0,0,236,0.0,0.0,-0.792687,-1.344075,-2.436871
2008-01-01 18:30:00,https://habrahabr.ru/post/18285/,habrahabr.ru,18285,@DezmASter,design,False,7344,"Сумасшедшие яйца, или сервис для отслеживания ...",1,1,353,1.0,0.0,-0.485214,-0.831946,-1.873193
2008-01-01 18:34:00,https://habrahabr.ru/post/18286/,habrahabr.ru,18286,@DezmASter,design,False,8431,"Сумасшедшие яйца, или сервис для отслеживания ...",47,72,1200,45.0,5.0,0.446090,1.794023,0.574656
2008-01-02 01:32:00,https://habrahabr.ru/post/18291/,habrahabr.ru,18291,@Taoorus,design,False,5662,"Сглаживание шрифтов, и субпиксельная отрисовка",102,36,5700,72.0,3.0,1.627469,1.300050,1.167375
2008-01-02 14:34:00,https://geektimes.ru/post/18294/,geektimes.ru,18294,@dennydo,NaN,False,3706,Почему мне не нравится iPhone,230,6,1400,73.0,15.0,1.362397,0.961094,2.020356
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016-10-31 21:45:00,https://habrahabr.ru/company/mailru/blog/314014/,habrahabr.ru,314014,NaN,develop,False,56033,Парсинг JSON — это минное поле,58,347,35500,164.0,6.0,1.870174,2.124862,1.258447
2016-10-31 22:44:00,https://geektimes.ru/post/282086/,geektimes.ru,282086,@marks,NaN,False,5504,Заработала система раннего предупреждения НАСА...,52,16,12900,22.0,0.0,-0.091464,-0.511814,0.465118
2016-10-31 23:26:00,https://geektimes.ru/post/282090/,geektimes.ru,282090,@Shapelez,NaN,False,2883,Арзамасский городской суд постановил заблокиро...,16,14,15800,28.0,4.0,0.198924,-0.656390,-0.408714


In [14]:
train_df_dates.index

DatetimeIndex(['2008-01-01 18:19:00', '2008-01-01 18:30:00',
               '2008-01-01 18:34:00', '2008-01-02 01:32:00',
               '2008-01-02 14:34:00', '2008-01-02 16:14:00',
               '2008-01-03 12:55:00', '2008-01-03 17:20:00',
               '2008-01-03 22:54:00', '2008-01-03 23:56:00',
               ...
               '2016-10-31 19:53:00', '2016-10-31 19:54:00',
               '2016-10-31 20:01:00', '2016-10-31 20:29:00',
               '2016-10-31 21:08:00', '2016-10-31 21:45:00',
               '2016-10-31 22:44:00', '2016-10-31 23:26:00',
               '2016-10-31 23:52:00', '2016-10-31 23:56:00'],
              dtype='datetime64[ns]', name='published', length=134137, freq=None)

In [15]:
train_df_dates["post_id"].resample('1y').count().idxmax()

Timestamp('2015-12-31 00:00:00', freq='A-DEC')

In [16]:
features = ['author', 'flow', 'domain','title']
train_size = int(0.7 * train_df.shape[0])
len(train_df), train_size
X, y = train_df.loc[:, features],  train_df['favs_lognorm'] #отделяем признаки от целевой переменной

X_test = test_df.loc[:, features]
X_train, X_valid = X.iloc[:train_size, :], X.iloc[train_size:,:]

y_train, y_valid = y.iloc[:train_size], y.iloc[train_size:]

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer_title =  TfidfVectorizer( min_df=3, max_df=0.3 ,ngram_range=(1, 3))

X_train_title = vectorizer_title.fit_transform(X_train['title'])
X_valid_title = vectorizer_title.transform(X_valid['title'])
X_test_title = vectorizer_title.transform(X_test['title'])

In [52]:
X_train_title.shape

(93895, 50624)

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer_title_char =  TfidfVectorizer( analyzer='char')

X_train_title_char = vectorizer_title_char.fit_transform(X_train['title'])
X_valid_title_char = vectorizer_title_char.transform(X_valid['title'])
X_test_title_char = vectorizer_title_char.transform(X_test['title'])
len(vectorizer_title_char.vocabulary_)

218

In [22]:
feats = ['author', 'flow', 'domain']
X_train[feats][:5]

,author,flow,domain
0,@Tapac,develop,habrahabr.ru
1,@DezmASter,design,habrahabr.ru
2,@DezmASter,design,habrahabr.ru
3,@Taoorus,design,habrahabr.ru
4,@dennydo,NaN,geektimes.ru


In [23]:
X_train[feats][:5].fillna('-').T.to_dict()

{0: {'author': '@Tapac', 'flow': 'develop', 'domain': 'habrahabr.ru'},
 1: {'author': '@DezmASter', 'flow': 'design', 'domain': 'habrahabr.ru'},
 2: {'author': '@DezmASter', 'flow': 'design', 'domain': 'habrahabr.ru'},
 3: {'author': '@Taoorus', 'flow': 'design', 'domain': 'habrahabr.ru'},
 4: {'author': '@dennydo', 'flow': '-', 'domain': 'geektimes.ru'}}

In [24]:
X_train[feats][:5].fillna('-').T.to_dict().values()


dict_values([{'author': '@Tapac', 'flow': 'develop', 'domain': 'habrahabr.ru'}, {'author': '@DezmASter', 'flow': 'design', 'domain': 'habrahabr.ru'}, {'author': '@DezmASter', 'flow': 'design', 'domain': 'habrahabr.ru'}, {'author': '@Taoorus', 'flow': 'design', 'domain': 'habrahabr.ru'}, {'author': '@dennydo', 'flow': '-', 'domain': 'geektimes.ru'}])

In [26]:
dict_vect = DictVectorizer()
dict_vect_matrix = dict_vect.fit_transform(X_train[feats][:5].fillna('-').T.to_dict().values())
dict_vect_matrix

<3x3 sparse matrix of type '<class 'numpy.float64'>'
	with 3 stored elements in Compressed Sparse Row format>

In [31]:
dict_vect_matrix.toarray()


array([[0., 0., 1., 0., 0., 1., 0., 0., 1.],
       [1., 0., 0., 0., 0., 1., 0., 1., 0.],
       [1., 0., 0., 0., 0., 1., 0., 1., 0.],
       [0., 1., 0., 0., 0., 1., 0., 1., 0.],
       [0., 0., 0., 1., 1., 0., 1., 0., 0.]])

In [32]:
dict_vect_matrix.shape


(5, 9)

In [33]:
for col in feats:
    print(col,len(X_train[col][:5].fillna('-').unique()))

author 4
flow 3
domain 2


In [34]:
dict_vect.feature_names_


['author=@DezmASter',
 'author=@Taoorus',
 'author=@Tapac',
 'author=@dennydo',
 'domain=geektimes.ru',
 'domain=habrahabr.ru',
 'flow=-',
 'flow=design',
 'flow=develop']

In [36]:
a = pd.DataFrame(dict_vect_matrix.toarray() , columns=dict_vect.feature_names_)
a

,author=@DezmASter,author=@Taoorus,author=@Tapac,author=@dennydo,domain=geektimes.ru,domain=habrahabr.ru,flow=-,flow=design,flow=develop
0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0
1,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
2,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
3,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
4,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0


In [44]:
def to_DictVectorizer_param(data , columns):
    return data[columns].fillna('-').T.to_dict().values()

In [45]:
vectorizer_feats = DictVectorizer()





X_train_feats = vectorizer_feats.fit_transform(to_DictVectorizer_param(X_train , feats) )
X_valid_feats = vectorizer_feats.transform(to_DictVectorizer_param(X_valid , feats))
X_test_feats = vectorizer_feats.transform(to_DictVectorizer_param(X_test , feats))


X_train_feats.shape

(93895, 17869)

In [40]:
X_train[feats].fillna('-').T.to_dict().values()

,author,flow,domain
0,@Tapac,develop,habrahabr.ru
1,@DezmASter,design,habrahabr.ru
2,@DezmASter,design,habrahabr.ru
3,@Taoorus,design,habrahabr.ru
4,@dennydo,NaN,geektimes.ru
...,...,...,...
93890,@YuriyIvon,develop,habrahabr.ru
93891,@marks,NaN,geektimes.ru
93892,NaN,develop,habrahabr.ru
93893,@marks,NaN,geektimes.ru


In [47]:
X_train_new = scipy.sparse.hstack([X_train_title, X_train_feats, X_train_title_char])
X_valid_new = scipy.sparse.hstack([X_valid_title, X_valid_feats, X_valid_title_char])
X_test_new =  scipy.sparse.hstack([X_test_title, X_test_feats, X_test_title_char])

In [ ]:
from sklearn.linear_model import Ridge
model1 = Ridge(alpha=1.0)
model1.fit(X_train_new, y_train)

train_preds1 = model1.predict(X_train_new)
valid_preds1 = model1.predict(X_valid_new)

print('Ошибка на трейне',mean_squared_error(y_train, train_preds1))
print('Ошибка на тесте',mean_squared_error(y_valid, valid_preds1))


In [51]:
model2 = Ridge(alpha=0.1)
model2.fit(X_train_new, y_train)

train_preds1 = model2.predict(X_train_new)
valid_preds1 = model2.predict(X_valid_new)

print('Ошибка на трейне',mean_squared_error(y_train, train_preds1))
print('Ошибка на тесте',mean_squared_error(y_valid, valid_preds1))

Ошибка на трейне 0.18454645052957042
Ошибка на тесте 0.903316400310199
